## 3. Implementierung der Hauptkomponentenanalyse

Wir beginnen zunächst mit einem schon bekannten Datensatz *Boston Housing*. Zur praktischen Berechnung der Hauptkomponentenanalyse gehen Sie folgt vor:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print(f"numpy version: {np.__version__}, pandas version: {pd.__version__}")

url     = 'https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
cols    = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B', 'LSTAT','TGT']
boston  = pd.read_csv(url, sep=' ', skipinitialspace=True, header=None, names=cols, index_col=False)

boston.head()

Ausschließen der Zielvariable `TGT` aus dem Datensatz, sowie katgorische Variablen (`CHAS`, `RAD`) ???

In [ ]:
boston = boston.drop(columns=['CHAS', 'RAD', 'TGT'])  # drop categorical and target variable
boston.head()

1. Gegeben eine Menge von $n$ $d$-dimensionalen Datenpunkten $\mathbf{x}_i$, berechnen Sie zuerst deren Mittelwert $\boldsymbol{\mu}_x = \frac{1}{n} \sum_{i=1}^n \mathbf{x}_i$ für jedes einzelne Merkmal und ziehen ihn von allen Datenpunkten ab (Zentrierung).


Mittels pandas `mean()` Funktion kann der Mittelwert für jedes Feature einfach berechnet werden.

In [ ]:
mean = boston.mean()
mean

Ebenso kann die Zentrierung des Datensatzes durch einfache Subtraktion des Mittelwerts von jedem Eintrag durchgeführt werden.

In [ ]:
boston_centered = boston - mean
boston_centered.head()

2. Normieren Sie dann alle Merkmale so, dass sie eine Varianz von 1 haben. Dieser Schritt ist optional, aber meist vorteilhaft.

In [ ]:
boston_normalized = boston_centered / boston.std()
boston_normalized.head()

Vor dem nächsten Schritt noch Prüfen oder der Mittelwert und die Zentrierung korrekt durchgeführt wurden. Der Mittelwert sollte jetzt für alle Features um die 0 liegen, und die Standardabweichung sollte 1 sein.

In [ ]:
mean_vals = boston_normalized.mean()
std_vals = boston_normalized.std()
print("Mittelwerte nach Normalisierung:\n", mean_vals)
print("Standardabweichungen nach Normalisierung:\n", std_vals)

3. Kopieren Sie alle $\mathbf{x}_i$ als Reihen in eine $n \times d$-Matrix $X$, die sog. Daten- oder Designmatrix.

In [ ]:
X = boston_normalized.to_numpy()
X.shape

4. Zur Lösung des Eigenwertproblens berechnen Sie die Singulärwertzerlegung von $X$ (z.B. mit `numpy.linalg.svd()`): $$ X = UDV^\top $$
Wer nicht weiß, was eine Singuärwertzerlegung ist oder macht, der lese bitte in den entsprechenden Wikipedia-Einträgen nach. Im Prinzip könnte man auch direkt die Eigenwerte der Kovarianzmatrix (s. Folie 12) berechnen (z.B. mit `numpy.linalg.eig()`), diese Methode ist aber meist aufwändiger und numerisch weniger stabil.

In [ ]:
U, D, Vt = np.linalg.svd(X)

Die Matrix $X$ ist definiert als:
$$
X \in \reals^{m \times n} \text{ mit Rang r}
$$
In unserem Fall also eine Matrix $X \in \reals^{506 \times 11}$ (ohne die Zielvariable und die kategorischen Variablen).

Die Matrix $U$ ist per Definition orthogonal und hat die Dimension $m \times m$. Das bedeutet, Ihre Inverse ist gleich ihrer Transponierten: $U^{-1} = U^\top$. Dies soll hier mal beispielhaft geprüft werden:

In [ ]:
U.shape

In [ ]:
U_inv = np.linalg.inv(U)
U_T = U.T
print(f"U_inv ist gleich U_T?: {np.allclose(U_inv, U_T)}")

Äquivalent dazu ist die Überprüfung, ob das Produkt von $U$ und $U^\top$ die Einheitsmatrix ergibt.

In [ ]:
prod = np.dot(U, U_T)
print(f"Produkt von U und U_T ist Einheitsmatrix?: {np.allclose(prod, np.eye(U.shape[0]))}")

Die Matrix $V$ ist ebenfalls orthogonal und hat die Dimension $n \times n$.

In [ ]:
Vt.shape

In [ ]:
# print Vt as matrix with its values to three decimal places
np.set_printoptions(precision=3, suppress=True, linewidth=100)
print(Vt)

Die Diagonalmatrix $D$ enthält die Singulärwerte von $X$ in absteigender Reihenfolge auf der Diagonalen und hat die Dimension $m \times n$.

In [ ]:
D.shape

In [ ]:
D

5. Die ersten $r$ Basisvektoren $\mathbf{q}_i$  (d.h die ersten $r$ Hauptkomponenten) sind die ersten $r$ Spalten der orthogonalen $d \times d$-Matrix $V$.

Die Hauptkomponenten sind also die ersten $r$ **Spalten (*columns*)** der Matrix $V$ oder alternativ der ersten $r$ **Zeilen (*rows*)** der Matrix $V^\top$. 

In [ ]:
n, d = X.shape

# set number of principal components to keep
r = 3

# get the first r columns of Vt (which correspond to the first r principal components)
Q = Vt.T[:, :r] # transpose Vt to get V, then take first r columns [row_min : row_max, col_min : col_max]
Q.shape

In [ ]:
print(Q)

6. Die Projektionen $a_i$ der Daten $\mathbf{x}_i$ auf die ersten $r$ Basisvektoren $\mathbf{q}_i$ (d.h die neuen Variablenwerte im neuen Koordinatensystem) sind die die ersten $r$ Spalten der $n \times d$-Matrix $UD$.

In [ ]:
scores = X @ Q  # project data onto first r principal components
scores.shape

In [ ]:
scores_alt = U[:, :r] * D[:r]  # alternative way using U and D
scores_alt.shape

In [ ]:
assert np.allclose(scores, scores_alt), "Projektionen stimmen nicht überein!"

7. Die Standardabweichungen entlang der Hauptkomponenten $\mathbf{q}_i$ sind die Diagonalelemente der Diagonalmatrix $D$ geteilt durch $\sqrt{n - 1}$.

In [ ]:
std_along_pcs = D[:r] / np.sqrt(n - 1)
std_along_pcs

In [ ]:
eigvals = (D**2) / (n - 1)
assert np.allclose(std_along_pcs, np.sqrt(eigvals[:r])), "Standardabweichungen stimmen nicht überein!"

Aufgaben:

a) Implementieren Sie ein Python-Modul, das eine Funktion zur Hauptkomponentenanalyse nach obigem Schema zur Verfügung stellt.

In [ ]:
import numpy as np
import pandas as pd

def pca(X:pd.DataFrame, r:int=3, normalize=True):
	# 1. calculate mean of data array and center
	mean = X.mean()
	X_centered = X - mean

	# 2. normalize if requested
	X_normalized = X_centered / X.std() if normalize else X_centered

	# 3. copy to data/designmatrix
	Xd = X_normalized.to_numpy()

	# 4. calculate singular value decomposition
	U, D, Vt = np.linalg.svd(Xd)
	V = Vt.T

	# 5. get r principal components
	Q = V[:, :r]

	# 6. project data onto first r principal components
	A = Xd @ Q

	# 7. calculate standard deviations along principal components
	std_pcs = D[:r] / np.sqrt(Xd.shape[0] - 1)

	return Q, A, std_pcs


b) Testen Sie Ihr Modul innerhalb eines IPython-Notebooks am Datensatz *Boston Housing*. Lassen Sie dabei die Variable `TGT` weg. Stellen Sie Ihre Ergebnisse in einer Tabelle mit den Eigenwerten der Kovarianzmatrix (Achtung: die Diagonalelemente von $D$ müssen dafür quadriert und durch n − 1 geteilt werden. Warum?), dem Anteil der zugehörigen Hauptkomponente an an der Gesamtvarianz (“erklärte Varianz”) und der kumulativen erklärten Varianz dar, d.h. welchen Varianzanteil die ersten $n$ Komponenten zusammen erklären. Wieviele Dimensionen können Sie weglassen, wenn Sie 10%, 5% und 1% Fehler bei der Dimensionsreduktion zulassen?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print(f"numpy version: {np.__version__}, pandas version: {pd.__version__}")

url     = 'https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
cols    = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B', 'LSTAT','TGT']
boston  = pd.read_csv(url, sep=' ', skipinitialspace=True, header=None, names=cols, index_col=False)

boston = boston.drop(columns=['TGT'])  # drop target variable
boston.head()

In [ ]:
Q, A, std_pcs = pca(boston, r=boston.shape[1], normalize=True)

In [ ]:
eigenvalues = (std_pcs ** 2) * (boston.shape[0] - 1)
explained_variance = eigenvalues / np.sum(eigenvalues)
cumulative_explained_variance = np.cumsum(explained_variance)
pca_results = pd.DataFrame({
    'Eigenvalue': eigenvalues,
    'Explained Variance': explained_variance,
    'Cumulative Explained Variance': cumulative_explained_variance
})

print(pca_results)


# Bestimmung der Anzahl der Dimensionen, die weggelassen werden können
def dimensions_to_retain(threshold):
    return np.argmax(cumulative_explained_variance >= (1 - threshold)) + 1


for error in [0.10, 0.05, 0.01]:
    dims = dimensions_to_retain(error)
    print(f"Dimensions to retain for {error * 100}% error: {dims}")

c) Berechnen Sie die Matrix der Korrelationskoeffizienten für die transformierten Variablen und interpretieren Sie das Ergebnis.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

corr_A = pd.DataFrame(A).corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr_A, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Korrelations-Heatmap der Hauptkomponenten')
plt.show()

**Interpretation des Ergebnisses:**
Wie in der Heatmap zu sehen sind die Diagonalelemente alle 1 (logisch, da jede Variable perfekt mit sich selbst korreliert ist) und alle anderen Elemente sind 0. Dies deutet darauf hin, dass die Hauptkomponenten unkorreliert sind und somit eine Reduktion der Dimensionalität ohne Informationsverlust möglich ist.

d) Berechnen Sie den Korrelationskoeffizienten der Projektionen auf die ersten drei Hauptkomponenten mit den ursprünglichen Variablen. Interpretieren Sie Ihr Ergebnis.

In [ ]:
boston_data = boston_normalized.copy()
PCs = pd.DataFrame(A[:,:3], columns=['PC1', 'PC2', 'PC3'])
combined = pd.concat([boston_data, PCs], axis=1)
corr_pc_orig = combined.corr().loc[PCs.columns, boston_data.columns]

plt.figure(figsize=(10,6))
sns.heatmap(corr_pc_orig, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Korrelations-Heatmap zwischen Hauptkomponenten und Originalvariablen')
plt.show()

e. Stellen Sie die ersten beiden der neuen Variablen als Scatterplot dar (am besten in Pandas-Dataframe importieren). Plotten Sie dabei alle Datenpunkte mit einem Hauspreis oberhalb des Medians aller Hauspreise in einer anderen Farbe als die Datenpunkte unterhalb. Eignen sich die beiden neuen Variablen zur Vorhersage des Hauspreises?